### Runpod 설정

In [1]:
import os

# 모델 저장 경로를 네트워크 볼륨(/workspace) 내부로 지정
os.environ["HF_HOME"] = "/workspace/hf_cache"

In [ ]:
# 필수 라이브러리 설치/업데이트
!pip install transformers accelerate bitsandbytes openai torch nltk huggingface_hub bert-score

In [3]:
from huggingface_hub import notebook_login
notebook_login()

---
### GPT

In [113]:
import time
import json
import openai

In [114]:
def generate_feedback_openai(model_name, prompt):
    start_time = time.perf_counter()
    ttft = 0
    generated_tokens = 0
    content = ""
    first_token_received = False
    
    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            response_format={"type": "json_object"},
            stream=True,                              # 토큰 단위로 수신
            stream_options={"include_usage": True}    # 마지막 chunk에 토큰 정보 포함
        )
        
        for chunk in response:
            # 첫번째 토큰이 들어오는 시점 확인 (ttft)
            if not first_token_received and chunk.choices and chunk.choices[0].delta.content:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            
            # 내용 누적
            if chunk.choices and chunk.choices[0].delta.content:
                content += chunk.choices[0].delta.content
            
            # 토큰 사용량 확인
            if chunk.usage is not None:
                generated_tokens = chunk.usage.completion_tokens
        
        end_time = time.perf_counter()
        total_duration = end_time - start_time
        
        # tps 계산 (생성된 토큰 수 / 전체 소요 시간)
        tps = generated_tokens / total_duration if total_duration > 0 else 0

        result_json = json.loads(content)
        
        metrics = {
                "ttft": round(ttft, 3),
                "tps": round(tps, 2),
                "total_tokens": generated_tokens
            }

        return result_json, metrics
    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}", "오류"

---

### sllm

In [6]:
import re
import json
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

In [21]:
def generate_summary_sllm(model_name, prompt):
    try:
        # 모델 및 토크나이저 로드 (기존 유지)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )
        
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        messages = [{"role": "system", "content": prompt}]
        inputs = tokenizer.apply_chat_template(
            messages, 
            tokenize=True, 
            add_generation_prompt=True, 
            return_tensors="pt"
        ).to(model.device)

        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        
        start_time = time.perf_counter()
        ttft = 0
        first_token_received = False
        full_content = ""

        generation_kwargs = dict(
            input_ids=inputs,
            streamer=streamer,
            max_new_tokens=1024,  # [수정] 512는 너무 짧아 잘릴 수 있으므로 1024로 확장
            repetition_penalty=1.2,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
        
        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()

        for new_text in streamer:
            if not first_token_received and len(new_text) > 0:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            full_content += new_text

        thread.join()
        end_time = time.perf_counter()
        
        total_duration = end_time - start_time
        generated_token_count = len(tokenizer.encode(full_content))
        tps = generated_token_count / total_duration if total_duration > 0 else 0

        # --- [JSON 추출 및 클리닝 파트 최적화] ---
        content = {}
        try:
            # 1. 앞뒤 잡설 및 마크다운 태그 제거
            cleaned_raw = re.sub(r'```json|```', '', full_content).strip()
            cleaned_raw = re.sub(r'//.*', '', cleaned_raw) # 주석 제거
            
            # 2. 첫 번째 '{' 부터 마지막 '}' 까지만 추출
            start_idx = cleaned_raw.find('{')
            end_idx = cleaned_raw.rfind('}')
            
            if start_idx != -1:
                # 만약 '}'가 없거나 형식이 깨졌을 경우를 대비해 잘린 부분 복구 시도
                if end_idx == -1 or end_idx < start_idx:
                    cleaned_raw = cleaned_raw[start_idx:] + '"}}' # 최소한의 닫는 괄호 강제 추가
                else:
                    cleaned_raw = cleaned_raw[start_idx:end_idx+1]
                
                # 3. JSON 파싱
                content = json.loads(cleaned_raw)
            else:
                content = {"error": "JSON 형식을 찾을 수 없음", "raw": full_content}
        except json.JSONDecodeError:
            # 파싱 실패 시, 따옴표가 안 닫힌 경우 등을 고려해 한 번 더 클리닝 시도
            try:
                # 따옴표나 괄호가 중간에 잘린 경우를 위한 강제 보정
                fixed_raw = cleaned_raw.strip()
                if not fixed_raw.endswith('}'): fixed_raw += '"}'
                if not fixed_raw.endswith('}'): fixed_raw += '}'
                content = json.loads(fixed_raw)
            except:
                content = {"error": "JSON 파싱 실패", "raw": full_content}
        # ------------------------------------------

        del model, tokenizer, inputs
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        metrics = {
            "ttft": round(ttft, 3),
            "tps": round(tps, 2),
            "total_tokens": generated_token_count
        }

        return content, metrics

    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}", None

### 테스트 코드

In [ ]:
import os
import json
import glob

# 파일 경로 설정
testset_dir = "./testsets"
json_files = glob.glob(os.path.join(testset_dir, "*.json"))

results_list = []

# 파일 루프 시작
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data_list = json.load(f)
        
    for item in data_list:
        # 데이터 추출
        script = item.get('consulting_content', "")
        summary_ref = item['instructions'][0]['data'][0]['output']

        print(f"테스트 실행 중: {item.get('source_id', 'Unknown ID')}")

        # 모델 테스트
        # prompt 생성
        system_prompt = f"""
        상담 스크립트를 평가 기준에 따라 객관적으로 평가하세요
        
        ### 제약 사항
        1. 모든 감점에는 구체적인 발화 근거를 반드시 제시한다
        2. 고객의 감사 표현은 고객 발화에서만 카운트한다
        3. 추측, 설명 문장, 자연어 해설 금지 - JSON만 출력한다
        4. 총 점수는 60점이며 각 2개 점수의 총합이다
        
        ### 상담 스크립트
        {script}
        
        ### 평가 기준
        1. 매뉴얼 준수 (50점에서 감점하는 방식)
        intro
        - 인사말
          0점: 첫인사 + 마무리 멘트 모두 수행
          -5점: 첫인사 또는 마무리 멘트 누락
        - 고객확인
          0점: 고객정보를 고객에게 직접 질문
          -5점: 상담원이 고객정보를 먼저 발화하여 정보 누출
        
        response
        - 호응어
          0점: 공감/감성 호응
          -5점: 기운 없음, 짜증 섞인 표현
        - 대기 표현
          0점: 대기 표현 모두 수행
          -5점: 대기 표현 누락
        
        explanation
        - 커뮤니케이션
          0점: 핵심 요약 + 이해 쉬운 설명
          -5점: 일방적 설명, 단답형
        - 알기 쉬운 설명
          0점: 고객 눈높이 설명 + 부연
          -5점: 복잡한 설명/상담자 관점 설명
        
        proactivity
        - 적극성
           0점: 적극적 대응
          -5점: 수동적 대응, 대안 없음
        - 언어표현
          0점: 정중/경어체/긍정 표현
          -5점: 전문용어, 줄임말, 명령조, 무시 표현
        
        accuracy
        - 정확한 업무처리
          0점: 오류 없음
          -10점: 임의 판단으로 업무 오류 발생
        
        2. 고객 감사 표현 (10점)
        - 고객 발화 중 감사/칭찬 키워드 포함 시 1회 카운트
        - 0회: 0점 / 1회: 5점 / 2회 이상: 10점
        
        
        ### 출력 형식 (JSON)
        {{
          "manual_compliance": {{
            "intro": {{
              "score": 0,
              "evidence": []
            }},
            "response": {{
              "score": 0,
              "evidence": []
            }},
            "explanation": {{
              "score": 0,
              "evidence": []
            }},
            "proactivity": {{
              "score": 0,
              "evidence": []
            }},
            "accuracy": {{
              "score": 0,
              "evidence": []
            }},
            "manual_score": "0~50점"
          }},
          "customer_thanks": {{
            "count": 0,
            "thanks_score": "0~10점",
            "evidence": []
          }},
          "final_score": {{
            "total": "manual_score + thanks_score",
            "feedback": ""
          }}
        }}
        """
        
        res, metrics = generate_summary_sllm("kakaocorp/kanana-nano-2.1b-instruct", system_prompt)
        # res, metrics = generate_summary_sllm("kakaocorp/kanana-1.5-8b-instruct-2505", system_prompt)
        # res, metrics = generate_feedback_openai("gpt-4.1-mini", system_prompt)
            
        # 결과 저장
        results_list.append({
            "id": item.get('source_id'),
            "res": res,
            "metrics": metrics,
        })

In [30]:
import os
import pandas as pd

df_new = pd.json_normalize(results_list)
df_new['model'] = "kanana-1.5-8b" 

output_file = "evaluation_results_feedback.csv"

if not os.path.exists(output_file):
    df_new.to_csv(output_file, index=False, mode='w', encoding='utf-8-sig')
else:
    df_new.to_csv(output_file, index=False, mode='a', encoding='utf-8-sig', header=False)

print(f"데이터가 추가되었습니다: {output_file}")

데이터가 추가되었습니다: evaluation_results_feedback.csv


---
### 후처리 시간 및 감정 전환 평가

In [ ]:
worktime = 100
emotion_set = ['부정', '긍정', '부정']

In [90]:
def evaluate_call(work_time, emotions):
    work_score = 0
    emotion_score = 0
    
    # 후처리 시간 평가
    time_indicator = 90
    
    if work_time <= time_indicator:
        work_score = 20
    elif work_time >= 90:
        if work_time < 120:
            work_score = 15
        elif work_time < 150:
            work_score = 10
        elif work_time < 200:
            work_score = 5
        else:
            work_score = 0
    
    # 감정 전환 평가
    def get_step_score(before, after):
        if before == after:
            return 3
        
        scores = {
            ("부정", "중립"): 5,
            ("부정", "긍정"): 10,
            ("중립", "긍정"): 10,
            ("중립", "부정"): 0,
            ("긍정", "부정"): 0,
            ("긍정", "중립"): 5
        }
        return scores.get((before, after), 0)

    # 단계별 점수 계산
    score_step_1 = get_step_score(emotions[0], emotions[1]) # 초반 -> 중반
    score_step_2 = get_step_score(emotions[1], emotions[2]) # 중반 -> 후반
    
    emotion_score = score_step_1 + score_step_2
    
    return {
        "work_score": work_score, 
        "emotion_score": emotion_score
        }

In [91]:
score = evaluate_call(worktime, emotion_set)
score

{'work_score': 15, 'emotion_score': 10}